In [219]:
from rtdl import MLP
from catboost import CatBoostClassifier
import optuna
import pytorch_lightning as pl 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
# import torchmetrics.functional as metric_f
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler

ModuleNotFoundError: No module named 'rtdl'

In [ ]:
%pip install rtdl -U



Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 609.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 549.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 825.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 23.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 77.6 MB/s eta 0:00:00


# Загрузка данных

In [ ]:
train_data = pd.read_csv("../data/my_ready_dataset.csv")
train_targets = pd.read_csv("../data/my_ready_targets.csv").astype(int)

In [ ]:
train_data.shape, train_targets.shape

In [ ]:
train_percent = 0.75
val_percent = 0.1

x_train_list = []
x_val_list = []
x_test_list = []

y_train_list = []
y_val_list = []
y_test_list = []

for key, data in train_data.groupby("station_id"):
    targets = train_targets[train_targets.station_id == key]
    data.sort_values("local_time", inplace=True)
    targets.sort_values("local_time", inplace=True)
    
    train_size = int(len(data) * train_percent)
    val_size = int(len(data) * val_percent)
    test_size = len(data) - train_size - val_size
     
    x_train_list.append(data.iloc[:train_size])
    y_train_list.append(targets.iloc[:train_size])
    
    x_val_list.append(data.iloc[train_size:train_size + val_size])
    y_val_list.append(targets.iloc[train_size:train_size + val_size])

    x_test_list.append(data.iloc[train_size + val_size:])
    y_test_list.append(targets.iloc[train_size + val_size:])

In [185]:
train_data = train_data.iloc[:-1]
train_targets = train_targets.iloc[1:]

In [186]:
X_train = train_data.iloc[:45000].drop(columns="local_time").values
y_train = train_targets.DOZD.iloc[:45000].values

In [187]:
train_targets

,VETER,SHKVAL,METEL,DOZD,SNEG,GRAD,TUMAN,GOLOLED
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
52162,0,0,0,0,1,0,0,0
52163,0,0,0,0,1,0,0,0
52164,0,0,0,0,1,0,0,0
52165,0,0,0,0,1,0,0,0


In [188]:
X_train.shape, y_train.shape

((45000, 810), (45000,))

In [189]:
X_val = X_train[40000:]
y_val = y_train[40000:]
X_train = X_train[:40000]
y_train = y_train[:40000]

In [190]:
X_test = train_data.iloc[45000:].drop(columns="local_time").values
y_test = train_targets.DOZD.iloc[45000:].values

# CatBoost

In [210]:
def objective(trial):
    params = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 10.0),
        "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.99, log=True)
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = CatBoostClassifier(**params)

    gbm.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=0
    )

    preds = gbm.predict(X_val)
    return f1_score(y_val, preds)

In [193]:
base_model = CatBoostClassifier()

In [201]:
base_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=0)

In [202]:
y_pred = model.predict(X_test)

In [209]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [203]:
f1_score(y_test, y_pred)

0.6384820239680427

In [204]:
np.mean(y_test == y_pred)

0.8484510186994139

In [205]:
feature_names = train_data.drop(columns="local_time").columns

In [206]:
fi = model.feature_importances_

In [207]:
top_features_idx = np.argsort(-fi)

In [211]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2022-12-11 16:48:03,962] A new study created in memory with name: no-name-f25f9cfd-0a4a-4740-b464-193c6dce2de8
/home/jupyter/.local/lib/python3.8/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-12-11 16:48:33,529] Trial 0 finished with value: 0.5270270270270271 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.016907444898176895, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 3.87169065570451, 'leaf_estimation_iterations': 5, 'learning_rate': 0.18792588538682062, 'bagging_temperature': 5.307041986921375}. Best is trial 0 with value: 0.5270270270270271.
[I 2022-12-11 16:48:55,171] Trial 1 finished with value: 0.661132075471698 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.029703890052226123, 'depth': 3, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'l2_leaf_reg': 0.18315046367718563, 'leaf_estimation_iterations': 4, 'learning_rate': 0.2958295088074001}. Best is trial 1 with value: 0.661132075471698.
[W 2022-12-11 16:59:22,686] Trial 2 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.8/site-packages/optuna/st

KeyboardInterrupt: 

# MLP

In [191]:
#!g1.1
class WeatherMLP(pl.LightningModule):
    def __init__(self, d_in, d_layers, dropout, d_out):
        super().__init__()
        self.model = MLP.make_baseline(d_in, d_layers, dropout, d_out)
        self.criterion = nn.CrossEntropyLoss()

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=1e-3)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch):
        x, y = batch
        y_pred = self(x)

        loss = self.criterion(y_pred, y)

        self.log({"train_loss": loss})

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)

        loss = self.criterion(y_pred, y)
        f1 = metric_f.f1(torch.argmax(y_pred, -1), y, task="binary").item()

        self.log({"val_loss": loss, "val_f1_score": f1}, progress_bar=True)

Failed to deserialize variable 'pl'. Run the following code to delete it:
  del_datasphere_variables('pl')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/usr/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/__init__.py", line 20, in <module>
    from pytorch_lightning import metrics  # n

NameError: name 'pl' is not defined

In [159]:
#!g1.1
callbacks = [
    pl.callbacks.ModelCheckpoint("checkpoints/mlp"),
    pl.callbacks.EarlyStopping("val_f1_score", mode="max")
]

trainer = pl.Trainer(callbacks=callbacks, gradient_clip_val=1.0, gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [160]:
#!g1.1
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [167]:
#!g1.1
def make_dataset(X, y):
    X = X.astype(float)
    dataset = []
    for i in range(len(X)):
        dataset.append((
            torch.from_numpy(X[i]).float(),
            y[i]
        ))
    return dataset

In [168]:
#!g1.1
train_dataset = make_dataset(X_train_scaled, y_train)
val_dataset = make_dataset(X_val_scaled, y_val)
test_dataset = make_dataset(X_test_scaled, y_test)

In [169]:
#!g1.1
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [170]:
#!g1.1
y_train[0]

0

In [171]:
#!g1.1
for batch in DataLoader(train_dataset, batch_size=2, shuffle=True):
    x, y = batch
    print(y)
    print(y.shape)
    break

tensor([0, 1])
torch.Size([2])


In [178]:
#!g1.1
trainer.fit(WeatherMLP(X_train_scaled.shape[1], [512, 512], 0.2, 2), train_loader, val_loader)

Validation sanity check: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | MLP              | 678 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
678 K     Trainable params
0         Non-trainable params
678 K     Total params
2.716     Total estimated model params size (MB)
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TypeError: f1() got an unexpected keyword argument 'task'

In [176]:
#!g1.1
dir(metric_f)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'accuracy',
 'auc',
 'audio',
 'auroc',
 'average_precision',
 'bert_score',
 'bleu_score',
 'calibration_error',
 'char_error_rate',
 'classification',
 'cohen_kappa',
 'confusion_matrix',
 'cosine_similarity',
 'dice_score',
 'embedding_similarity',
 'explained_variance',
 'f1',
 'fbeta',
 'hamming_distance',
 'hinge',
 'image',
 'image_gradients',
 'iou',
 'kl_divergence',
 'matthews_corrcoef',
 'mean_absolute_error',
 'mean_absolute_percentage_error',
 'mean_squared_error',
 'mean_squared_log_error',
 'pairwise',
 'pairwise_cosine_similarity',
 'pairwise_euclidean_distance',
 'pairwise_linear_similarity',
 'pairwise_manhatten_distance',
 'pearson_corrcoef',
 'pesq',
 'pit',
 'pit_permutate',
 'precision',
 'precision_recall',
 'precision_recall_curve',
 'psnr',
 'r2_score',
 'recall',
 'regression',
 'retrieval',
 'retrieval_average_precision',

In [ ]:
#!g1.1
